Для початку зробимо невеличкий препроцесінг дати, дуже невеличкий

In [67]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
df = pd.read_csv('marketing_data-28347077.csv', parse_dates = ['registration_date'], index_col=0)
df['logins_all'] = df['logins_1_lt'] + df['logins_3_lt'] + df['logins_7_lt']
df['percent_of_logins'] = df['logins_all'] / df['registrations']
df['loses_per_person'] = (df['revenue_7_days'] - df['users_acqusition_costs']) / df['registrations']
for i in range(len(df.index)):
  if df['percent_of_logins'][i] > 1:
    df['percent_of_logins'][i] = 1
  if df['users_acqusition_costs'][i] < 0:
    df['users_acqusition_costs'][i].drop()

<ipython-input-67-c6d9777646b2>:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [59]:
df

,traffic_type,id_partner,age_group,device,country_tier,registrations,users_acqusition_costs,revenue_7_days,logins_1_lt,logins_3_lt,logins_7_lt,logins_all,percent_of_logins,loses_per_person
registration_date,,,,,,,,,,,,,,
2021-06-24,Google Search,1,35-,iOS,Tier_1,60,220.0,2.99,5,3,1,9,0.150000,-3.616833
2021-09-13,Google Search,1,35-,MacOS,Tier_1,8,0.0,0.00,0,0,0,0,0.000000,0.000000
2021-07-12,Google Search,1,35-55,Android,Tier_1,121,2550.0,225.97,28,19,11,58,0.479339,-19.206860
2021-03-26,Google Search,1,35-55,iOS,Tier_1,95,1900.0,351.92,23,9,2,34,0.357895,-16.295579
2021-04-26,Google Search,1,35-,Android,Tier_1,51,915.0,256.09,7,2,2,11,0.215686,-12.919804
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-09,Google Media,6,35-55,Other,Tier_1,15,90.0,0.00,1,1,1,3,0.200000,-6.000000
2021-07-19,Google Media,6,55+,Other,Tier_1,23,65.5,0.00,5,0,2,7,0.304348,-2.847826
2021-02-26,Google Media,6,55+,Other,Tier_1,29,194.0,0.00,2,3,2,7,0.241379,-6.689655


**За якими показниками можна виміряти ефективність закупівлі кожним з партнерів? Які партнери є найкращими, найгіршими за цими показниками?**

На мою думку, найкращим для цього показником є відсоток людей, що поврнулись на платформу за якийсь час

Надамо оверол оцінку, подивишись який середній відсоток людей взагалі повертається від кожного партнера

In [42]:
df['percent_of_logins'].groupby(df['id_partner']).mean()

id_partner
1    0.394518
2    0.373605
3    0.382921
4    0.353956
5    0.224035
6    0.240781
Name: percent_of_logins, dtype: float64

Як можна помітити, партнери Google Search і Facebook мають майже однаковий відсоток користувачів, які повертались на платформу, а обидва партнери Google Media показують незадовільні результати.


In [41]:
df['registrations'].groupby(df['id_partner']).sum() * df['percent_of_logins'].groupby(df['id_partner']).mean()

id_partner
1     85717.404772
2     60509.763150
3    119028.365945
4    152741.126371
5    167897.571544
6    201590.541504
dtype: float64

Але при тому ж маємо цікаве спостреження, що партнери Google Media принесли найбільшу кількість людей на платформу, які повертались через якийсь час після реєстрації

In [40]:
df['loses_per_person'].groupby(df['id_partner']).mean()


id_partner
1   -12.910695
2   -17.801047
3    -8.296703
4    -8.716916
5    -2.126468
6    -2.458638
Name: loses_per_person, dtype: float64

І маємо ще одне спостереження: на те, щоб привести людину, не треба платити багато грошей Google Media, бо вони або беруть мало коштів за роботу, або користувачі, що вони принесли, платять активно компанії.

Тож оверол можна скзати, що за сукупністю всіх факторів, Google Media з її обидвами партнерами є найкращими постачальниками користувачів в нашу компанію, а Google Search навпаки найгіршими, хоч і відсоток повернень у людей, які перейшли через них є найвищим, але ціна і сумарна кількість запрошених людей не є оптимальним.

**Чи існує залежність між періодом закупки у часі і її ефективністю по кожному з партнерів? Чи є інші фактори, що впливають на ефективність закупки?**

На цьому етапі нам треба видалити всі nan рядки в стовпчику з віком, хоч їх і всього 2

А також сортуємо індекси

Ну і малюємо графік щоб побачити різницю

In [69]:
df['age_group'].dropna()
df.sort_index()
df = df.reset_index()
df.sort_values(by='registration_date')
#графік з ефективністю
fig = px.histogram(
    df,
    x='registration_date',
    y='percent_of_logins',
    facet_col='id_partner',
    facet_col_wrap=2,
    color='id_partner',
    width=800,
)
fig.show()

Бачимо, що партнери 1 і 6 є трохи стабільними протягом року, 2 приносить якісних користувачів тільки під початок року, поітм спад, 3 навпаки, пі дкінець року все більше приносить, 4 і 5 мають маленькі скачки в кінці і початку року віпдовідно

In [72]:
#графік з кількістю
fig = px.scatter(
    df,
    x='registration_date',
    y='registrations',
    facet_col='id_partner',
    facet_col_wrap=2,
    color='id_partner',
    width=800,
)
fig.show()

Як бачимо, партнер 1 рівномірно заоохочує користувачів, 2 має невеликий скачок під цінець року, 5 має невеликий скачок в останніій третині року, 6 має суттєвий скачок в среедині року, 5 має величезний скачок в перші третитні року, 6 доволі стабільний.

**Яка аудиторія є найбільш цільовою для компанії? Надайте рекомендації партнерам щодо того,яку аудиторію їм варто закуповувати**




In [77]:
fig = px.histogram(df, x="age_group", color="id_partner")
fig.update_layout(barmode='group')
fig.show()

**Як між джерелами трафіку і партнерами варто розподілити нові маркетингові бюджети?**